# Alan's Benchmarking Report Pages

In [1]:
import pickle
import pandas as pd
import numpy as np
from importlib import reload
import sys

# bench_util module is in the parent directory, because it will be part of
# the production code.  Add the parent directory to the Path so Python can
# import from there
sys.path.insert(0, '../')   
import bench_util as bu

## Unpickle the Dataframe for Preprocessed Utility Data & Make Utility Function Object

In [2]:
# Unpickle the pre-processed DataFrame
df = pickle.load(open('dfu3.pkl', 'rb'))

# Unpickle the raw utility bill DataFrame, which is needed below to make
# the utility function object.
df_raw = pickle.load(open('df_raw.pkl', 'rb'))

df.head()  # the processed data

,site_id,service_type,cal_year,cal_mo,item_desc,units,cost,days_served,usage,fiscal_year,fiscal_mo,mmbtu
0,03,Electricity,2010,7,Electricity charge,-,24.137931,35.0,0.000000,2011,1,0.000000
1,03,Electricity,2010,7,Electricity charge,kWh,1304.274655,9.5,6656.551724,2011,1,22.712154
2,03,Electricity,2010,8,Electricity charge,-,39.498433,62.0,0.000000,2011,2,0.000000
3,03,Electricity,2010,8,Electricity charge,kWh,4940.646254,31.0,26822.236155,2011,2,91.517470
4,03,Electricity,2010,9,Electricity charge,-,16.363636,27.0,0.000000,2011,3,0.000000


In [3]:
# this is only needed to update any code changes I may have made
# since last importing the module above.
reload(bu)     

# Make an object that has the various utility functions.
# The object needs access to the raw utility bill DataFrame and the spreadsheet
# containing other application data.
ut = bu.Util(df_raw, '../data/Other_Building_Data.xlsx')

## Set Site ID for All of the Reports Below

In [4]:
site = 'ANSBG1'

## DataFrame for "Energy Index Comparison" Report, Page 2

In [5]:
# Need to have this constant at the top of the script in order to know what
# Fiscal Year to use in the Report.
FISCAL_YEAR_FOR_ONE_YEAR_REPORT = 2016

### Table 1

### Table 2

In [7]:
# Filter down to just the records of the targeted fiscal year
df1 = df.query('fiscal_year == @FISCAL_YEAR_FOR_ONE_YEAR_REPORT')
df1.head()

,site_id,service_type,cal_year,cal_mo,item_desc,units,cost,days_served,usage,fiscal_year,fiscal_mo,mmbtu
78,03,Electricity,2015,7,Electricity charge,-,39.091459,62.0,0.000000,2016,1,0.000000
79,03,Electricity,2015,7,Electricity charge,kWh,3603.339259,31.0,16614.193548,2016,1,56.687628
80,03,Electricity,2015,8,Electricity charge,-,15.679374,25.0,0.000000,2016,2,0.000000
81,03,Electricity,2015,8,Electricity charge,kWh,4567.802303,68.0,23701.950202,2016,2,80.871054
82,03,Electricity,2015,9,Electricity charge,kWh,5757.846701,90.0,31256.845683,2016,3,106.648357


In [12]:
# Get Total Utility cost by building. This includes non-energy utilities as well.
df2 = df1.pivot_table(index='site_id', values=['cost'], aggfunc=np.sum)
df2.columns = ['cost_total']
df2.head()

,cost_total
site_id,
03,110978.515317
04,117368.777340
05,536715.062940
06,111150.278132
07,114241.225387


In [13]:
# Save this into the Final DataFrame that we will build up as we go.
df_final = df2.copy()

In [15]:
# Get a list of the Energy Services and restrict the data to
# just these services
energy_svcs = bu.missing_energy_services([])
df2 = df1.query('service_type == @energy_svcs')
df2.service_type.unique()

array(['Electricity', 'Natural Gas', 'Oil #1', 'Steam'], dtype=object)

In [19]:
# Summarize Cost by Service Type
df3 = pd.pivot_table(df2, index='site_id', columns='service_type', values='cost', aggfunc=np.sum)
df3.head()

service_type,Electricity,Natural Gas,Oil #1,Steam
site_id,,,,
03,72445.857903,2694.708115,25462.23,NaN
04,73894.233046,1810.998276,27422.11,NaN
05,329872.481124,NaN,NaN,152059.040143
06,66238.678564,1649.382308,24443.21,NaN
07,77881.893809,5273.385696,12884.15,NaN


In [20]:
# Change column names
cols = ['cost_{}'.format(bu.change_name(col)) for col in df3.columns]
df3.columns = cols
df3.head()

,cost_electricity,cost_natural_gas,cost_fuel_oil,cost_district_heat
site_id,,,,
03,72445.857903,2694.708115,25462.23,NaN
04,73894.233046,1810.998276,27422.11,NaN
05,329872.481124,NaN,NaN,152059.040143
06,66238.678564,1649.382308,24443.21,NaN
07,77881.893809,5273.385696,12884.15,NaN


In [21]:
# Add a total energy cost column
df3['cost_total_energy'] = df3.sum(axis=1)
df3.head()

,cost_electricity,cost_natural_gas,cost_fuel_oil,cost_district_heat,cost_total_energy
site_id,,,,,
03,72445.857903,2694.708115,25462.23,NaN,100602.796018
04,73894.233046,1810.998276,27422.11,NaN,103127.341322
05,329872.481124,NaN,NaN,152059.040143,481931.521267
06,66238.678564,1649.382308,24443.21,NaN,92331.270872
07,77881.893809,5273.385696,12884.15,NaN,96039.429504


In [22]:
# Add a total Heat Cost Column
df3['cost_total_heat'] = df3.cost_total_energy.fillna(0.0) - df3.cost_electricity.fillna(0.0)
df3.head()

,cost_electricity,cost_natural_gas,cost_fuel_oil,cost_district_heat,cost_total_energy,cost_total_heat
site_id,,,,,,
03,72445.857903,2694.708115,25462.23,NaN,100602.796018,28156.938115
04,73894.233046,1810.998276,27422.11,NaN,103127.341322,29233.108276
05,329872.481124,NaN,NaN,152059.040143,481931.521267,152059.040143
06,66238.678564,1649.382308,24443.21,NaN,92331.270872,26092.592308
07,77881.893809,5273.385696,12884.15,NaN,96039.429504,18157.535696


In [23]:
# Add this to the final DataFrame
df_final = pd.concat([df_final, df3], axis=1)
df_final.head()

,cost_total,cost_electricity,cost_natural_gas,cost_fuel_oil,cost_district_heat,cost_total_energy,cost_total_heat
03,110978.515317,72445.857903,2694.708115,25462.23,NaN,100602.796018,28156.938115
04,117368.777340,73894.233046,1810.998276,27422.11,NaN,103127.341322,29233.108276
05,536715.062940,329872.481124,NaN,NaN,152059.040143,481931.521267,152059.040143
06,111150.278132,66238.678564,1649.382308,24443.21,NaN,92331.270872,26092.592308
07,114241.225387,77881.893809,5273.385696,12884.15,NaN,96039.429504,18157.535696


In [24]:
# Summarize MMBtu by Service Type
df3 = pd.pivot_table(df2, index='site_id', columns='service_type', values='mmbtu', aggfunc=np.sum)
df3.head()

service_type,Electricity,Natural Gas,Oil #1,Steam
site_id,,,,
03,1262.055985,100.094903,2455.920,NaN
04,1364.923930,55.896000,2122.605,NaN
05,6072.938690,NaN,NaN,10792.771429
06,1213.149257,47.834710,1914.705,NaN
07,1408.801323,229.159452,1021.815,NaN


In [25]:
# Change column names
cols = ['mmbtu_{}'.format(bu.change_name(col)) for col in df3.columns]
df3.columns = cols
df3.head()

,mmbtu_electricity,mmbtu_natural_gas,mmbtu_fuel_oil,mmbtu_district_heat
site_id,,,,
03,1262.055985,100.094903,2455.920,NaN
04,1364.923930,55.896000,2122.605,NaN
05,6072.938690,NaN,NaN,10792.771429
06,1213.149257,47.834710,1914.705,NaN
07,1408.801323,229.159452,1021.815,NaN


In [26]:
# Add a total mmbtu column
df3['mmbtu_total'] = df3.sum(axis=1)
df3.head()

,mmbtu_electricity,mmbtu_natural_gas,mmbtu_fuel_oil,mmbtu_district_heat,mmbtu_total
site_id,,,,,
03,1262.055985,100.094903,2455.920,NaN,3818.070888
04,1364.923930,55.896000,2122.605,NaN,3543.424930
05,6072.938690,NaN,NaN,10792.771429,16865.710119
06,1213.149257,47.834710,1914.705,NaN,3175.688967
07,1408.801323,229.159452,1021.815,NaN,2659.775775


In [27]:
# Add a total Heat mmbtu Column
df3['mmbtu_total_heat'] = df3.mmbtu_total.fillna(0.0) - df3.mmbtu_electricity.fillna(0.0)
df3.head()

,mmbtu_electricity,mmbtu_natural_gas,mmbtu_fuel_oil,mmbtu_district_heat,mmbtu_total,mmbtu_total_heat
site_id,,,,,,
03,1262.055985,100.094903,2455.920,NaN,3818.070888,2556.014903
04,1364.923930,55.896000,2122.605,NaN,3543.424930,2178.501000
05,6072.938690,NaN,NaN,10792.771429,16865.710119,10792.771429
06,1213.149257,47.834710,1914.705,NaN,3175.688967,1962.539710
07,1408.801323,229.159452,1021.815,NaN,2659.775775,1250.974452


In [28]:
# Add this to the final DataFrame
df_final = pd.concat([df_final, df3], axis=1)
df_final.head()

,cost_total,cost_electricity,cost_natural_gas,cost_fuel_oil,cost_district_heat,cost_total_energy,cost_total_heat,mmbtu_electricity,mmbtu_natural_gas,mmbtu_fuel_oil,mmbtu_district_heat,mmbtu_total,mmbtu_total_heat
03,110978.515317,72445.857903,2694.708115,25462.23,NaN,100602.796018,28156.938115,1262.055985,100.094903,2455.920,NaN,3818.070888,2556.014903
04,117368.777340,73894.233046,1810.998276,27422.11,NaN,103127.341322,29233.108276,1364.923930,55.896000,2122.605,NaN,3543.424930,2178.501000
05,536715.062940,329872.481124,NaN,NaN,152059.040143,481931.521267,152059.040143,6072.938690,NaN,NaN,10792.771429,16865.710119,10792.771429
06,111150.278132,66238.678564,1649.382308,24443.21,NaN,92331.270872,26092.592308,1213.149257,47.834710,1914.705,NaN,3175.688967,1962.539710
07,114241.225387,77881.893809,5273.385696,12884.15,NaN,96039.429504,18157.535696,1408.801323,229.159452,1021.815,NaN,2659.775775,1250.974452


In [30]:
# Electricity kWh summed by building
df3 = pd.pivot_table(df2.query('units == "kWh"'), index='site_id', values='usage', aggfunc=np.sum)
df3.columns = ['electricity_kWh']
df3.head()

,electricity_kWh
site_id,
03,3.698875e+05
04,4.000363e+05
05,1.779877e+06
06,3.555537e+05
07,4.128961e+05


In [31]:
# Include in Final DF
df_final = pd.concat([df_final, df3], axis=1)
df_final.head()

,cost_total,cost_electricity,cost_natural_gas,cost_fuel_oil,cost_district_heat,cost_total_energy,cost_total_heat,mmbtu_electricity,mmbtu_natural_gas,mmbtu_fuel_oil,mmbtu_district_heat,mmbtu_total,mmbtu_total_heat,electricity_kWh
03,110978.515317,72445.857903,2694.708115,25462.23,NaN,100602.796018,28156.938115,1262.055985,100.094903,2455.920,NaN,3818.070888,2556.014903,3.698875e+05
04,117368.777340,73894.233046,1810.998276,27422.11,NaN,103127.341322,29233.108276,1364.923930,55.896000,2122.605,NaN,3543.424930,2178.501000,4.000363e+05
05,536715.062940,329872.481124,NaN,NaN,152059.040143,481931.521267,152059.040143,6072.938690,NaN,NaN,10792.771429,16865.710119,10792.771429,1.779877e+06
06,111150.278132,66238.678564,1649.382308,24443.21,NaN,92331.270872,26092.592308,1213.149257,47.834710,1914.705,NaN,3175.688967,1962.539710,3.555537e+05
07,114241.225387,77881.893809,5273.385696,12884.15,NaN,96039.429504,18157.535696,1408.801323,229.159452,1021.815,NaN,2659.775775,1250.974452,4.128961e+05


In [33]:
# Electricity kW, both Average and Max by building
df3 = pd.pivot_table(df2.query('units == "kW"'), index='site_id', values='usage', aggfunc=[np.mean, np.max])
df3.columns = ['kW_average', 'kW_max']
df3.head()

,kW_average,kW_max
site_id,,
ASLELC1,116.112001,141.213739
ASLELC2,136.252236,176.485000
BALHHW,78.165155,101.244118
BAOBG1,185.840038,206.610091
BHPSKI2,46.528571,59.254372


In [34]:
# Add into Final Frame
df_final = pd.concat([df_final, df3], axis=1)
df_final.head()

,cost_total,cost_electricity,cost_natural_gas,cost_fuel_oil,cost_district_heat,cost_total_energy,cost_total_heat,mmbtu_electricity,mmbtu_natural_gas,mmbtu_fuel_oil,mmbtu_district_heat,mmbtu_total,mmbtu_total_heat,electricity_kWh,kW_average,kW_max
03,110978.515317,72445.857903,2694.708115,25462.23,NaN,100602.796018,28156.938115,1262.055985,100.094903,2455.920,NaN,3818.070888,2556.014903,3.698875e+05,NaN,NaN
04,117368.777340,73894.233046,1810.998276,27422.11,NaN,103127.341322,29233.108276,1364.923930,55.896000,2122.605,NaN,3543.424930,2178.501000,4.000363e+05,NaN,NaN
05,536715.062940,329872.481124,NaN,NaN,152059.040143,481931.521267,152059.040143,6072.938690,NaN,NaN,10792.771429,16865.710119,10792.771429,1.779877e+06,NaN,NaN
06,111150.278132,66238.678564,1649.382308,24443.21,NaN,92331.270872,26092.592308,1213.149257,47.834710,1914.705,NaN,3175.688967,1962.539710,3.555537e+05,NaN,NaN
07,114241.225387,77881.893809,5273.385696,12884.15,NaN,96039.429504,18157.535696,1408.801323,229.159452,1021.815,NaN,2659.775775,1250.974452,4.128961e+05,NaN,NaN


In [37]:
# Check a building that has a kW charge
df_final.loc['BALHHW']

cost_total              73620.449647
cost_electricity        70905.957742
cost_natural_gas                 NaN
cost_fuel_oil                    NaN
cost_district_heat               NaN
cost_total_energy       70905.957742
cost_total_heat             0.000000
mmbtu_electricity        1445.775015
mmbtu_natural_gas                NaN
mmbtu_fuel_oil                   NaN
mmbtu_district_heat              NaN
mmbtu_total              1445.775015
mmbtu_total_heat            0.000000
electricity_kWh        423732.419355
kW_average                 78.165155
kW_max                    101.244118
Name: BALHHW, dtype: float64

In [40]:
# Add in Square footage and DD Weather Site info
df_bldg = ut.building_info_df()[['sq_ft', 'dd_site']]
# Add into Final Frame
df_final = pd.concat([df_final, df_bldg], axis=1)
df_final.head()

,cost_total,cost_electricity,cost_natural_gas,cost_fuel_oil,cost_district_heat,cost_total_energy,cost_total_heat,mmbtu_electricity,mmbtu_natural_gas,mmbtu_fuel_oil,mmbtu_district_heat,mmbtu_total,mmbtu_total_heat,electricity_kWh,kW_average,kW_max,sq_ft,dd_site
03,110978.515317,72445.857903,2694.708115,25462.23,NaN,100602.796018,28156.938115,1262.055985,100.094903,2455.920,NaN,3818.070888,2556.014903,3.698875e+05,NaN,NaN,10123,PAFA
04,117368.777340,73894.233046,1810.998276,27422.11,NaN,103127.341322,29233.108276,1364.923930,55.896000,2122.605,NaN,3543.424930,2178.501000,4.000363e+05,NaN,NaN,10123,PAFA
05,536715.062940,329872.481124,NaN,NaN,152059.040143,481931.521267,152059.040143,6072.938690,NaN,NaN,10792.771429,16865.710119,10792.771429,1.779877e+06,NaN,NaN,10123,PAFA
06,111150.278132,66238.678564,1649.382308,24443.21,NaN,92331.270872,26092.592308,1213.149257,47.834710,1914.705,NaN,3175.688967,1962.539710,3.555537e+05,NaN,NaN,10123,PAFA
07,114241.225387,77881.893809,5273.385696,12884.15,NaN,96039.429504,18157.535696,1408.801323,229.159452,1021.815,NaN,2659.775775,1250.974452,4.128961e+05,NaN,NaN,10123,PAFA


## DataFrame for "Utility Cost Overview" Report, Page 3

In [8]:
# From the main DataFrame, get only the rows for this site, and only get
# the needed columns for this analysis
df1 = df.query('site_id == @site')[['service_type', 'fiscal_year', 'fiscal_mo', 'cost']]
df1.head()

,service_type,fiscal_year,fiscal_mo,cost
17200,Electricity,2011,1,602.300
17201,Electricity,2011,1,0.000
17202,Electricity,2011,1,699.360
17203,Electricity,2011,2,1212.035
17204,Electricity,2011,2,0.000


In [9]:
df2 = pd.pivot_table(
    df1, 
    values='cost', 
    index=['fiscal_year'], 
    columns=['service_type'],
    aggfunc=np.sum
)
df2

service_type,Electricity,Natural Gas,Sewer,Water
fiscal_year,,,,
2011,33894.895909,110858.416500,4716.576852,4379.291667
2012,41994.697841,94490.242371,5484.028148,5090.420833
2013,36961.451595,79970.329194,5169.324500,4783.854500
2014,35843.822155,48967.142581,4840.116833,4375.768167
2015,38017.419167,47128.677258,4593.016167,4156.942333
2016,29639.448485,45647.201628,6108.227500,5113.362500
2017,21849.234848,26682.420469,3326.440000,2875.330000


In [10]:
# Add in columns for the missing services
missing_services = bu.missing_services(df2.columns)
bu.add_columns(df2, missing_services)
df2

service_type,Electricity,Natural Gas,Sewer,Water,Steam,Oil #1,Refuse
fiscal_year,,,,,,,
2011,33894.895909,110858.416500,4716.576852,4379.291667,0.0,0.0,0.0
2012,41994.697841,94490.242371,5484.028148,5090.420833,0.0,0.0,0.0
2013,36961.451595,79970.329194,5169.324500,4783.854500,0.0,0.0,0.0
2014,35843.822155,48967.142581,4840.116833,4375.768167,0.0,0.0,0.0
2015,38017.419167,47128.677258,4593.016167,4156.942333,0.0,0.0,0.0
2016,29639.448485,45647.201628,6108.227500,5113.362500,0.0,0.0,0.0
2017,21849.234848,26682.420469,3326.440000,2875.330000,0.0,0.0,0.0


In [11]:
# Add a Total column that sums the other columns
df2['Total'] = df2.sum(axis=1)
df2

service_type,Electricity,Natural Gas,Sewer,Water,Steam,Oil #1,Refuse,Total
fiscal_year,,,,,,,,
2011,33894.895909,110858.416500,4716.576852,4379.291667,0.0,0.0,0.0,153849.180928
2012,41994.697841,94490.242371,5484.028148,5090.420833,0.0,0.0,0.0,147059.389193
2013,36961.451595,79970.329194,5169.324500,4783.854500,0.0,0.0,0.0,126884.959788
2014,35843.822155,48967.142581,4840.116833,4375.768167,0.0,0.0,0.0,94026.849736
2015,38017.419167,47128.677258,4593.016167,4156.942333,0.0,0.0,0.0,93896.054925
2016,29639.448485,45647.201628,6108.227500,5113.362500,0.0,0.0,0.0,86508.240113
2017,21849.234848,26682.420469,3326.440000,2875.330000,0.0,0.0,0.0,54733.425317


In [12]:
# Add a percent change column
df2['pct_change'] = df2.Total.pct_change()
df2

service_type,Electricity,Natural Gas,Sewer,Water,Steam,Oil #1,Refuse,Total,pct_change
fiscal_year,,,,,,,,,
2011,33894.895909,110858.416500,4716.576852,4379.291667,0.0,0.0,0.0,153849.180928,NaN
2012,41994.697841,94490.242371,5484.028148,5090.420833,0.0,0.0,0.0,147059.389193,-0.044133
2013,36961.451595,79970.329194,5169.324500,4783.854500,0.0,0.0,0.0,126884.959788,-0.137186
2014,35843.822155,48967.142581,4840.116833,4375.768167,0.0,0.0,0.0,94026.849736,-0.258960
2015,38017.419167,47128.677258,4593.016167,4156.942333,0.0,0.0,0.0,93896.054925,-0.001391
2016,29639.448485,45647.201628,6108.227500,5113.362500,0.0,0.0,0.0,86508.240113,-0.078681
2017,21849.234848,26682.420469,3326.440000,2875.330000,0.0,0.0,0.0,54733.425317,-0.367304


In [13]:
# Add in degree days
months_present = bu.months_present(df1)
deg_days = ut.degree_days_yearly(months_present, site)
df2['hdd'] = deg_days
df2

service_type,Electricity,Natural Gas,Sewer,Water,Steam,Oil #1,Refuse,Total,pct_change,hdd
fiscal_year,,,,,,,,,,
2011,33894.895909,110858.416500,4716.576852,4379.291667,0.0,0.0,0.0,153849.180928,NaN,13802
2012,41994.697841,94490.242371,5484.028148,5090.420833,0.0,0.0,0.0,147059.389193,-0.044133,12393
2013,36961.451595,79970.329194,5169.324500,4783.854500,0.0,0.0,0.0,126884.959788,-0.137186,12171
2014,35843.822155,48967.142581,4840.116833,4375.768167,0.0,0.0,0.0,94026.849736,-0.258960,13802
2015,38017.419167,47128.677258,4593.016167,4156.942333,0.0,0.0,0.0,93896.054925,-0.001391,12393
2016,29639.448485,45647.201628,6108.227500,5113.362500,0.0,0.0,0.0,86508.240113,-0.078681,12171
2017,21849.234848,26682.420469,3326.440000,2875.330000,0.0,0.0,0.0,54733.425317,-0.367304,12356


In [14]:
# Add in a column to show the numbers of months present for each year
# This will help to identify partial years.
mo_count = bu.month_count(months_present)
df2['month_count'] = mo_count
df2

service_type,Electricity,Natural Gas,Sewer,Water,Steam,Oil #1,Refuse,Total,pct_change,hdd,month_count
fiscal_year,,,,,,,,,,,
2011,33894.895909,110858.416500,4716.576852,4379.291667,0.0,0.0,0.0,153849.180928,NaN,13802,12
2012,41994.697841,94490.242371,5484.028148,5090.420833,0.0,0.0,0.0,147059.389193,-0.044133,12393,12
2013,36961.451595,79970.329194,5169.324500,4783.854500,0.0,0.0,0.0,126884.959788,-0.137186,12171,12
2014,35843.822155,48967.142581,4840.116833,4375.768167,0.0,0.0,0.0,94026.849736,-0.258960,13802,12
2015,38017.419167,47128.677258,4593.016167,4156.942333,0.0,0.0,0.0,93896.054925,-0.001391,12393,12
2016,29639.448485,45647.201628,6108.227500,5113.362500,0.0,0.0,0.0,86508.240113,-0.078681,12171,12
2017,21849.234848,26682.420469,3326.440000,2875.330000,0.0,0.0,0.0,54733.425317,-0.367304,12356,9


In [15]:
# Reverse the DataFrame
df2.sort_index(ascending=False, inplace=True)
df2

service_type,Electricity,Natural Gas,Sewer,Water,Steam,Oil #1,Refuse,Total,pct_change,hdd,month_count
fiscal_year,,,,,,,,,,,
2017,21849.234848,26682.420469,3326.440000,2875.330000,0.0,0.0,0.0,54733.425317,-0.367304,12356,9
2016,29639.448485,45647.201628,6108.227500,5113.362500,0.0,0.0,0.0,86508.240113,-0.078681,12171,12
2015,38017.419167,47128.677258,4593.016167,4156.942333,0.0,0.0,0.0,93896.054925,-0.001391,12393,12
2014,35843.822155,48967.142581,4840.116833,4375.768167,0.0,0.0,0.0,94026.849736,-0.258960,13802,12
2013,36961.451595,79970.329194,5169.324500,4783.854500,0.0,0.0,0.0,126884.959788,-0.137186,12171,12
2012,41994.697841,94490.242371,5484.028148,5090.420833,0.0,0.0,0.0,147059.389193,-0.044133,12393,12
2011,33894.895909,110858.416500,4716.576852,4379.291667,0.0,0.0,0.0,153849.180928,NaN,13802,12


In [16]:
# Export the rows of the DataFrame into a list of dictionaries for use
# in the template.  Rename columns to match template variables.
row_dicts = bu.df_to_dictionaries(df2, change_names={'Natural Gas': 'natural_gas', 'Oil #1': 'fuel_oil'})
row_dicts

[{'Electricity': 21849.234848484841,
  'Refuse': 0.0,
  'Sewer': 3326.4400000000001,
  'Steam': 0.0,
  'Total': 54733.42531723484,
  'Water': 2875.3299999999995,
  'fiscal_year': 2017,
  'fuel_oil': 0.0,
  'hdd': 12356.0,
  'month_count': 9.0,
  'natural_gas': 26682.420468749999,
  'pct_change': -0.36730390947936598},
 {'Electricity': 29639.448484848483,
  'Refuse': 0.0,
  'Sewer': 6108.2274999999991,
  'Steam': 0.0,
  'Total': 86508.240112872678,
  'Water': 5113.3625000000011,
  'fiscal_year': 2016,
  'fuel_oil': 0.0,
  'hdd': 12171.0,
  'month_count': 12.0,
  'natural_gas': 45647.201628024202,
  'pct_change': -0.078680779696023495},
 {'Electricity': 38017.419166666674,
  'Refuse': 0.0,
  'Sewer': 4593.0161666666672,
  'Steam': 0.0,
  'Total': 93896.054924731172,
  'Water': 4156.9423333333343,
  'fiscal_year': 2015,
  'fuel_oil': 0.0,
  'hdd': 12393.0,
  'month_count': 12.0,
  'natural_gas': 47128.677258064497,
  'pct_change': -0.0013910368310102728},
 {'Electricity': 35843.8221551724

In [17]:
# Put results into the final report dictionary that will be passed to the Template.
# This assumes you have created a couple graphs with the file names shown below.
pg3 = dict(rows=row_dicts, 
           g1='output/images/{}_pg2_g1.png'.format(site), 
           g2='output/images/{}_pg2_g2.png'.format(site)
      )
pg3

{'g1': 'output/images/ANSBG1_pg2_g1.png',
 'g2': 'output/images/ANSBG1_pg2_g2.png',
 'rows': [{'Electricity': 21849.234848484841,
   'Refuse': 0.0,
   'Sewer': 3326.4400000000001,
   'Steam': 0.0,
   'Total': 54733.42531723484,
   'Water': 2875.3299999999995,
   'fiscal_year': 2017,
   'fuel_oil': 0.0,
   'hdd': 12356.0,
   'month_count': 9.0,
   'natural_gas': 26682.420468749999,
   'pct_change': -0.36730390947936598},
  {'Electricity': 29639.448484848483,
   'Refuse': 0.0,
   'Sewer': 6108.2274999999991,
   'Steam': 0.0,
   'Total': 86508.240112872678,
   'Water': 5113.3625000000011,
   'fiscal_year': 2016,
   'fuel_oil': 0.0,
   'hdd': 12171.0,
   'month_count': 12.0,
   'natural_gas': 45647.201628024202,
   'pct_change': -0.078680779696023495},
  {'Electricity': 38017.419166666674,
   'Refuse': 0.0,
   'Sewer': 4593.0161666666672,
   'Steam': 0.0,
   'Total': 93896.054924731172,
   'Water': 4156.9423333333343,
   'fiscal_year': 2015,
   'fuel_oil': 0.0,
   'hdd': 12393.0,
   'month

## Convert the Notebook into a Script

In [18]:
# Convert the notebook to a script.
#!jupyter nbconvert --to script alan_report_pages.ipynb